In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import os
from keras.models import load_model
import cv2
import numpy as np 
from keras.layers import Input, Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import expand_dims
from keras.utils import load_img
from keras.utils import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras.optimizers import Adam
# load the image


def LoadCitraTraining(sDir,LabelKelas):  
  JumlahKelas=len(LabelKelas)
  TargetKelas = np.eye(JumlahKelas)
  # Menyiapkan variabel list untuk data menampung citra dan data target
  X=[]#Menampung Data Citra
  T=[]#Menampung Target
  for i in range(len(LabelKelas)):    
    #Membaca file citra di setiap direktori data set  
    DirKelas = os.path.join(sDir, LabelKelas[i])
    files = os.listdir(DirKelas)
    for f in files:
      ff=f.lower()  
      print(f)
      #memilih citra dengan extensi jpg,jpeg,dan png
      if (ff.endswith('.jpg')|ff.endswith('.jpeg')|ff.endswith('.png')):
         NmFile = os.path.join(DirKelas,f) 
         img= np.double(cv2.imread(NmFile,1))
         img=cv2.resize(img,(200,300));
         img= np.asarray(img)/255;
         img=img.astype('float32')
         #Menambahkan citra dan target ke daftar
         X.append(img)
         T.append(TargetKelas[i])
     #--------akhir loop :Pfor f in files-----------------
  #-----akhir  loop :for i in range(len(LabelKelas))----
  
  #Mengubah List Menjadi numppy array
  X=np.array(X)
  T=np.array(T)
  X=X.astype('float32')
  T=T.astype('float32')
  return X,T

def ModelDeepLearningCNN(JumlahKelas):
    input_img = Input(shape=(300,200, 3)) 
    # Convolutional Layer 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    # Convolutional Layer 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    # Convolutional Layer 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    # Convolutional Layer 4
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Dropout(0.25)(x)

    # Flatten and Dense Layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(JumlahKelas, activation='softmax')(x)

    ModelCNN = Model(input_img, x) 
    adam_optimizer = Adam(learning_rate=0.0001) 
    ModelCNN.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
    
    return ModelCNN

def TrainingCNN(JumlahEpoh,DirektoriDataSet,LabelKelas,NamaFileBobot):
    #Membaca Data training dan label Kelas 
    X,D=LoadCitraTraining(DirektoriDataSet,LabelKelas)
    JumlahKelas = len(LabelKelas)
    #Membuat Model CNN
    ModelCNN = ModelDeepLearningCNN(JumlahKelas)
    #Trainng
    history= ModelCNN.fit(X, D,epochs=JumlahEpoh,shuffle=True)
    #Menyimpan hasil learning
    ModelCNN.save(NamaFileBobot)
    #Mengembalikan output 
    return ModelCNN,history


def Klasifikasi(DirDataSet,DirKlasifikasi,LabelKelas,ModelCNN=[]):
#Menyiapkan Data input Yang akan di kasifikasikan
  X=[]
  ls = [];
  DirKelas = DirDataSet+"/"+DirKlasifikasi
  print(DirKelas)
  files = os.listdir(DirKelas)
  n=0;
  for f in files:
      ff=f.lower()  
      print(f)
      if (ff.endswith('.jpg')|ff.endswith('.jpeg')|ff.endswith('.png')):
         ls.append(ff) 
         NmFile = os.path.join(DirKelas,f)
         img= cv2.imread(NmFile,1)
         img=cv2.resize(img,(200,300))
         img= np.asarray(img)/255
         img=img.astype('float32')
         X.append(img)
     #----Akhir if-------------
  #---Akhir For 
  X=np.array(X)
  X=X.astype('float32')
  #Melakukan prediksi Klasifikasi
  hs=ModelCNN.predict(X)
  
  LKlasifikasi=[];
  LKelasCitra =[];
  n = X.shape[0]
  for i in range(n):
      v=hs[i,:]
      if v.max()>0.6:
          idx = np.max(np.where( v == v.max()))
          LKelasCitra.append(LabelKelas[idx])
      else:
          idx=-1
          LKelasCitra.append("-")
      #------akhir if
      LKlasifikasi.append(idx);
  #----akhir for
  LKlasifikasi = np.array(LKlasifikasi)
  return ls, hs, LKelasCitra

def LoadModel(sf):
  ModelCNN=load_model(sf)   
  return ModelCNN 

def ImageAugmentation(SPath,Kelas):
  parent_dir = SPath 
  directory = Kelas
  directoryExt =directory +"_ext"
  sdirExt = os.path.join(parent_dir, directoryExt)
  if not os.path.exists(sdirExt):
    os.mkdir(sdirExt)
  directory_image =directory
  sDir = os.path.join(parent_dir, directory_image)
  files = os.listdir(sDir)
  ii=0
  for f in files:
    ff=f.lower()
    if (ff.endswith('.jpg')|ff.endswith('.jpeg')|ff.endswith('.png')):
      print(ff)
      sfs= os.path.join(sDir,ff)
      img = load_img(sfs)
      img2 = np.array(img)
      sfn= os.path.join(sdirExt, ff)
      cv2.imwrite(sfn,img2)
      data = img_to_array(img)
      samples = expand_dims(data, 0)
      # width shift untuk menggeser kekiri atau kekanan, height shift untuk menggeser keatas atau kebawah
      datagen = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,brightness_range=[0.1,0.5],validation_split=0.3)
      it = datagen.flow(samples, batch_size=1)
      for i in range(9):
        batch = it.next()
        image = batch[0].astype('uint8')
        now = datetime.now() 
        ii=ii+1
        sf = now.strftime("%Y%m%d%H%M%S")+"_"+str(ii)+".jpg"
        sfn= os.path.join(sdirExt, sf)
        cv2.imwrite(sfn,image)



In [3]:
LabelKelas= [
    "2 Hearts", "3 Hearts", "4 Hearts", "5 Hearts", "6 Hearts", "7 Hearts", "8 Hearts", "9 Hearts", "10 Hearts", "Jack Hearts", "Queen Hearts", "King Hearts", "Ace Hearts",
    "2 Spades", "3 Spades", "4 Spades", "5 Spades", "6 Spades", "7 Spades", "8 Spades", "9 Spades", "10 Spades", "Jack Spades", "Queen Spades", "King Spades", "Ace Spades",
    "2 Diamonds", "3 Diamonds", "4 Diamonds", "5 Diamonds", "6 Diamonds", "7 Diamonds", "8 Diamonds", "9 Diamonds", "10 Diamonds", "Jack Diamonds", "Queen Diamonds", "King Diamonds", "Ace Diamonds",
    "2 Clubs", "3 Clubs", "4 Clubs", "5 Clubs", "6 Clubs", "7 Clubs", "8 Clubs", "9 Clubs", "10 Clubs", "Jack Clubs", "Queen Clubs", "King Clubs", "Ace Clubs"
] 
DirektoriDataSet=r"C:\Users\HP\Documents\PCV\2"
JumlahEpoh = 37
FileBobot = "Poker4.h5"
ModelCNN,history = TrainingCNN(JumlahEpoh,DirektoriDataSet,LabelKelas ,FileBobot)
ModelCNN.summary()



2H10_jpg.rf.896779deea30ef31ff1079fa9aad580b.jpg
2H10_jpg.rf.92a62a1424b28cea0846ea223db59b21.jpg
2H10_jpg.rf.d1b5331a3134464fd260ae1965835855.jpg
2H11_jpg.rf.00c432b18ef9368385c5938550093913.jpg
2H11_jpg.rf.02eca05e6345dc218de1cf93c3c4378f.jpg
2H11_jpg.rf.f8f9b98d930556d31e6687295bfbcf54.jpg
2H12_jpg.rf.015f60944c2912aec33676fe70360864.jpg
2H12_jpg.rf.036abb125bed46ac5ffb8a469548f241.jpg
2H12_jpg.rf.923a81d18c445d7d5d302bf2c921a4ef.jpg
2H14_jpg.rf.875a47db6a0d85118525b71f78575f91.jpg
2H14_jpg.rf.9f05e508147a828979839b2de6595851.jpg
2H14_jpg.rf.feda5cb8f9f0153296c9ca04be34221f.jpg
2H15_jpg.rf.4886a2a3617879d2677daef2c2e536b5.jpg
2H15_jpg.rf.624bd2ef953a8ac08402cc475f221658.jpg
2H15_jpg.rf.a5525ef8abc2afbd516da6921fa7f951.jpg
2H16_jpg.rf.a445beefeda89b0bab25532545807484.jpg
2H16_jpg.rf.a9781b73126ccd92c4fab4d97c061dec.jpg
2H16_jpg.rf.cba2d2152e31b2300d8635ddf33e85b4.jpg
2H17_jpg.rf.28e340656fcba7b587da3195f5ce5d47.jpg
2H5_jpg.rf.1c643fd15ee2b8143961e6af39bda77c.jpg
2H6_jpg.rf.c3f25f0105